In [ ]:
#드라이버 경로, 검색 키워드, 마지막 페이지 수동으로 설정 필요



import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
from tqdm import tqdm  # tqdm 불러오기

# 데이터프레임 초기화
data = {
    'Title_Text': [],  # 뉴스 제목
    'Article_Text': [],  # 뉴스 본문
    'DateTime': [],  # 작성 시간
    'Media_Title': [],  # 언론사
}

# 드라이버 경로와 URL 설정
path = 'C:/Temp/chromedriver_114/chromedriver.exe'
url = 'https://finance.naver.com/news/news_search.naver'

# 웹드라이버 실행
driver = webdriver.Chrome()
driver.get(url)

# 1. 검색 입력란 클릭 후 '하이브' 입력
search_box = driver.find_element(By.CSS_SELECTOR, "#contentarea_left > form > div > div > div > input.inputTxt")
search_box.click()
search_box.send_keys('하이브')

# 2. 선택자 클릭
option_label = driver.find_element(By.CSS_SELECTOR, "#contentarea_left > form > div > div > dl > dd:nth-child(2) > span:nth-child(3) > label")
option_label.click()

# 3. 날짜 설정
start_date_input = driver.find_element(By.CSS_SELECTOR, "#contentarea_left > form > div > div > dl > dd:nth-child(4) > span:nth-child(6) > label > input:nth-child(1)")
start_date_input.click()
start_date_input.clear()
new_start_date = "2024-01-01"
start_date_input.send_keys(new_start_date)

end_date_input = driver.find_element(By.CSS_SELECTOR, "#contentarea_left > form > div > div > dl > dd:nth-child(4) > span:nth-child(6) > label > input:nth-child(3)")
end_date_input.click()
end_date_input.clear()
new_end_date = "2024-10-14"
end_date_input.send_keys(new_end_date)

# 5. 검색 버튼 클릭
search_button = driver.find_element(By.CSS_SELECTOR, "#contentarea_left > form > div > div > div > input[type=image]:nth-child(2)")
search_button.click()

try:
    # 현재 페이지 번호 변수 설정
    current_page = 1
    max_page = 92  # 최대 페이지 수 설정, 수동으로 확인하는것이 필요함

    while current_page <= max_page:
        print(f"{current_page}페이지에서 링크 수집 및 방문 중...")

        # 현재 페이지에서 모든 articleSubject 링크 요소 가져오기
        article_links = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.newsList .articleSubject a'))
        )

        # tqdm을 사용하여 링크 방문 진행도 표시
        for i in tqdm(range(len(article_links)), desc=f'페이지 {current_page} 진행 상황'):
            article_links = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.newsList .articleSubject a'))
            )
            
            href = article_links[i].get_attribute('href')
            print(f"링크로 이동: {href}")
            
            # 링크 클릭 (링크로 직접 이동)
            driver.get(href)

            # 랜덤한 대기 시간 설정 (2초에서 5초 사이)
            random_sec = random.uniform(2, 4)
            time.sleep(random_sec)
            
            try:
                # 1. 제목 텍스트 추출
                try:
                    title_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.media_end_head_title span'))
                    )
                    title_text = title_element.text
                    print('제목:', title_text)
                except Exception as e:
                    print('제목을 찾을 수 없습니다.', e)
                    title_text = None

                # 2. 뉴스 본문 추출
                try:
                    article_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '#newsct_article'))
                    )
                    article_text = article_element.text
                    print('Article Text:', article_text)
                except Exception as e:
                    print('newsct_article 요소를 찾을 수 없습니다.', e)
                    article_text = None

                # 3. 작성 시간 추출
                try:
                    datetime_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'span.media_end_head_info_datestamp_time'))
                    )
                    datetime_text = datetime_element.text
                    print('날짜와 시간:', datetime_text)
                except Exception as e:
                    print('datetime 요소를 찾을 수 없습니다.', e)
                    datetime_text = None

                # 4. 언론사 추출
                try:
                    image_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'img.media_end_head_top_logo_img'))
                    )
                    media_title = image_element.get_attribute('title')
                    if not media_title:
                        raise AttributeError("title 속성 없음")
                    print("미디어 제목: ", media_title)
                except Exception as e:
                    print(f"오류 발생 또는 title 속성이 없습니다: {e}")
                    media_title = None

                # 추출한 데이터를 데이터프레임에 저장
                data['Title_Text'].append(title_text)
                data['Article_Text'].append(article_text)
                data['DateTime'].append(datetime_text)
                data['Media_Title'].append(media_title)

            except Exception as e:
                print("요소가 존재하지 않습니다. 뒤로 돌아갑니다.", e)
                driver.back()
                time.sleep(3)
                continue  # 다음 링크로 이동
            

            # 다시 원래 페이지로 돌아가기
            driver.back()

            # 랜덤 대기 시간 설정 (1초에서 3초 사이)
            random_sec = random.uniform(1, 3)
            time.sleep(random_sec)

        # 페이지 번호 증가
        current_page += 1

        # '다음' 버튼 클릭을 위해 현재 페이지가 10의 배수일 때 '다음' 클릭
        if current_page % 10 == 1 and current_page != 1:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'td.pgR a'))
                )
                next_button.click()

                # 랜덤 대기 시간 설정 (3초에서 6초 사이)
                random_sec = random.uniform(3, 6)
                time.sleep(random_sec)

            except Exception as e:
                print(f"'다음' 버튼 클릭 오류: {e}")
                break

        # 그 외에는 다음 페이지 번호 클릭
        else:
            try:
                next_page_link = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, f'td.on + td a'))
                )
                next_page_link.click()

                # 랜덤 대기 시간 설정 (2초에서 4초 사이)
                random_sec = random.uniform(2, 4)
                time.sleep(random_sec)

            except Exception as e:
                print(f"페이지 이동 오류: {e}")
                break

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 크롤링 작업이 완료되면 데이터프레임으로 변환
    df = pd.DataFrame(data)
    print(df.head())  # 데이터프레임 출력 (확인용)
    
    # 필요한 경로에 파일로 저장
    file_path = r'C:/Users/USER/Desktop/news_data.csv'
    df.to_csv(file_path, "news_data_셀트리온.csv", index=False, encoding='utf-8-sig')  # CSV 파일로 저장

    # 드라이버 종료
    driver.quit()



#####